In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from connectivity.curves import CURVES
from connectivity.load import get_h5_names_of_patient, MultipleHDFResponseLoader
from connectivity.analyze import find_max_n_replications, filter_logs

%load_ext autoreload
%autoreload 2
%aimport connectivity

load_dotenv()
base_path = os.getenv("BASE_PATH_PAPER", "/default/path")

### Demographics

- number of participants
- median IQR age

In [2]:
demographics_df = pd.read_csv(f"{base_path}/participant_demographics.csv")
allowed_ids = ["EL019", "EL020", "EL021", "EL022", "EL026", "EL027", "EL028"]

responses_df = pd.read_json(f"../publish_data/paper/significant_responses/response_channels_lf.json", orient="records")
responses_df = responses_df[responses_df["patient_id"].isin(allowed_ids)]

patient_ids = responses_df["patient_id"].unique()

print(f"Number of participants: {len(patient_ids)}. {patient_ids}")

ages = []
for patient_id in patient_ids:
    ages.append(demographics_df.loc[demographics_df["patient_id"] == patient_id, "age"].squeeze())

median_age = np.median(ages)
q75_age, q25_age = np.percentile(ages, [75, 25])
iqr_age = q75_age - q25_age

print(f"Age: median {median_age:.3f}, IQR: {iqr_age:.3f} [{q25_age: .3f} - {q75_age: .3f}]")

Number of participants: 7. ['EL026' 'EL020' 'EL019' 'EL028' 'EL021' 'EL027' 'EL022']
Age: median 28.000, IQR: 21.000 [ 25.000 -  46.000]


### Stimulations

- median of stimulations
- number of stimulation in mesio-temporal structures, number in frontal, number in pariental-occipital regions

In [8]:
from collections import Counter

# how to find number of stimulations per patient?
# create MRL, get logs and filter them as in significant responses
n_replications = 12
n_stims = []
isis_s = []
stimulation_regions = []
n_recording_channels = []
n_stims = []

rows = []
for patient_id in patient_ids:
    print(f"{pd.Timestamp.now()}: Processing p {patient_id}")
    names_h5 = get_h5_names_of_patient(base_path, patient_id, protocol="CR", new_overview_format=True)

    path_lookup = f"{base_path}/{patient_id}/Electrodes/Lookup.xlsx"
    paths_h5 = [
        f"{base_path}/{patient_id}/Electrophy/{name}.h5" for name in names_h5
    ]
    paths_logs = [
        f"{base_path}/{patient_id}/out/{name}_logs.csv" for name in names_h5
    ]
    # if CLEAN_DATA:
    #     path_excluded_responses = f"{base_path}/{patient_id}/{CLEAN_DATA_FILE}"
    # else:
    #     path_excluded_responses = None

    mrl = MultipleHDFResponseLoader(
        paths_h5=paths_h5,
        paths_logs=paths_logs,
        recording_names=names_h5,
        path_lookup=path_lookup,
        #path_excluded_responses=path_excluded_responses,
    )
    #if sleep_stage is not None and len(sleep_stage) > 0:
    #    mrl.add_sleep_score_to_logs()

    logs = mrl.get_logs()

    io_stim_channels = logs[logs["type"] == "CR_IO"][
        ["name_pos", "name_neg"]
    ].drop_duplicates()
    io_stim_channel_names = io_stim_channels.agg("-".join, axis=1).tolist()
    io_stim_channel_paths = mrl.get_channel_paths_from_names(io_stim_channel_names)
    print(io_stim_channel_names)
    n_stims_per_patient = []
    for chosen_stim_channel, io_stim_channel in io_stim_channels.iterrows():
        stim_channel_name = io_stim_channel["name_pos"] + "-" + io_stim_channel["name_neg"]
        stimulation_regions.append(mrl.get_regions_from_names(channel_names=[stim_channel_name])[0])

        _, max_n_replications = find_max_n_replications(
            complete_logs=logs,
            selected_stim_channel_name_neg=io_stim_channel["name_neg"],
            selected_stim_channel_name_pos=io_stim_channel["name_pos"],
            stim_protocol="CR_IO",
            sleep_states = []#sleep_states=sleep_stage,
        )
        if max_n_replications < n_replications:
            print(f"Not enough replications for {patient_id} - {io_stim_channel}")
            continue

        ## PREPARATION

        io_stimlist = filter_logs(
            complete_logs=logs,
            n_replications=n_replications,
            selected_stim_channel_name_pos=io_stim_channel["name_pos"],
            selected_stim_channel_name_neg=io_stim_channel["name_neg"],
            sleep_stages=[],#sleep_stages=sleep_stage,
            triplet_protocol=None
        )
        
        rows.append({
            "patient_id": patient_id,
            "stim_channel_name": stim_channel_name,
            "label": mrl.get_labels_from_names(channel_names=[stim_channel_name])[0],
            "regions": mrl.get_regions_from_names(channel_names=[stim_channel_name])[0]
        })

        isis_s.extend(io_stimlist["ISI_s"])
        n_stims.append(len(io_stimlist))

        channel_paths = mrl.get_channel_paths(
            exclude_stim_channels=True,
            exclude_noisy_channels=True,
            stim_channel_name_pos=io_stim_channel["name_pos"],
            stim_channel_name_neg=io_stim_channel["name_neg"],
            exclude_wm_only_channels=True,
            exclude_out_channels=True,
        )
        n_recording_channels.append(len(channel_paths))

        io_intensities = (
            logs[logs["type"] == "CR_IO"]["Int_prob"].drop_duplicates().tolist()
        )
        io_intensities.sort()
        #print(io_intensities)
print(f"ISI: mean {np.mean(isis_s):.2f}, std {np.std(isis_s):.2f}")
print(f"Recording channels: median {np.median(n_recording_channels)}, std {np.std(n_recording_channels)}")
stimulation_region_counts = Counter(stimulation_regions)
# Display counts
print(stimulation_region_counts)

df = pd.DataFrame(rows)
print(df)

2025-08-31 11:10:51.859745: Processing p EL026
['aI-L-5-aI-L-6', 'aH-L-5-aH-L-6', 'OFC-L-11-OFC-L-12', 'aH-L-1-aH-L-2', 'OFC-L-1-OFC-L-2']
Excluded 1 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 1 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 1 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 1 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Not enough replications for EL026 - name_pos    OFC-L-1
name_neg    OFC-L-2
Name: 12700, dtype: object
2025-08-31 11:10:52.949749: Processing p EL020
['L-BTa-1-L-BTa-2', 'L-Ia-1-L-Ia-2', 'L-Ha-1-L-Ha-2', 'L-A-1-L-A-2']
Excluded 2 noisy channels.
Excluded 15 WM-only channels.
Excluded 2 noisy channels.
Excluded 15 WM-only channels.
Excluded 2 noisy channels.
Excluded 15 WM-only channels.
Excluded 2 noisy channels.
Excluded 15 WM-only channels.
2025-08-31

c:\Users\E1024679\Documents\coherence\connectivity\load.py:703: UserWarning: Not all recordings have the same channel names. We use the first recording as a reference and keep only channels that are available in all recording. Missing channels in recording EL022_20230315_02h33m13_CR: ['traces/bipolar/lead/aH-L/aH-L-3-aH-L-4', 'traces/bipolar/lead/aH-L/aH-L-4-aH-L-5', 'traces/bipolar/lead/aH-L/aH-L-5-aH-L-6', 'traces/bipolar/lead/aH-L/aH-L-7-aH-L-8', 'traces/bipolar/lead/aH-L/aH-L-8-aH-L-9', 'traces/bipolar/lead/aH-L/aH-L-9-aH-L-10']
  warnings.warn(


['EntG-R-1-EntG-R-2', 'aH-L-1-aH-L-2', 'EntG-L-2-EntG-L-3', 'OF-L-1-OF-L-2']
Excluded 11 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 11 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 11 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
Excluded 11 noisy channels.
Excluded 25 WM-only channels.
Excluded 4 out/putament/ventricle channels.
ISI: mean 4.54, std 0.25
Recording channels: median 77.5, std 13.445468093186566
Counter({'Mesiotemporal': 15, 'Insula': 6, 'Orbitofrontal': 4, 'Dorsofrontal': 2, 'Basotemporal': 1, 'Superotemporal': 1})
   patient_id  stim_channel_name        label         regions
0       EL026      aI-L-5-aI-L-6          SIG          Insula
1       EL026      aH-L-5-aH-L-6  CollatS_ant    Basotemporal
2       EL026  OFC-L-11-OFC-L-12       IFGtri    Dorsofrontal
3       EL026      aH-L-1-aH-L-2         HIPP   Mesiotemporal
4

In [10]:
print("Total stimulations", np.sum(n_stims))
print(len(n_stims))

Total stimulations 5664
28
